In [ ]:
import pandas as pd
import numpy as np

In [ ]:
PM_2_5 = pd.read_csv('PM_25_24.csv', low_memory=False)
CO = pd.read_csv('CO_24.csv', low_memory=False)
O3 = pd.read_csv('Ozone_24.csv', low_memory=False)
NO2 = pd.read_csv('NO2_24.csv', low_memory=False)
SO2 = pd.read_csv('SO2_24.csv', low_memory=False)


In [ ]:
# Filter sample durations
PM_2_5 = PM_2_5[PM_2_5['Sample Duration'].isin(['24-HR BLK AVG', '24 HOUR'])]
PM_2_5.loc[PM_2_5['State Name'] == 'Idaho', 'Sample Duration'] = '24 HOUR'

CO = CO[CO['Sample Duration'] == '8-HR RUN AVG END HOUR']
SO2 = SO2[SO2['Sample Duration'] == '1 HOUR']

In [ ]:
# Keep only relevant columns
pm_cols = ['Date Local','Site Num','State Code','County Code','POC','Arithmetic Mean','AQI','Latitude','Longitude','State Name','City Name','County Name','Local Site Name']
co_cols = ['Date Local','Site Num','State Code','County Code','POC','1st Max Value','AQI']
o3_cols = ['Date Local','Site Num','State Code','County Code','POC','1st Max Value','AQI']
no2_cols = ['Date Local','Site Num','State Code','County Code','POC','1st Max Value','AQI']
so2_cols = ['Date Local','Site Num','State Code','County Code','POC','1st Max Value','AQI']

PM_2_5 = PM_2_5[pm_cols].copy()
CO = CO[co_cols].copy()
O3 = O3[o3_cols].copy()
NO2 = NO2[no2_cols].copy()
SO2 = SO2[so2_cols].copy()

In [167]:
# Rename columns for clarity
PM_2_5.rename(columns={'Arithmetic Mean':'PM25','AQI':'AQI_PM25'}, inplace=True)
CO.rename(columns={'1st Max Value':'CO','AQI':'AQI_CO'}, inplace=True)
O3.rename(columns={'1st Max Value':'O3','AQI':'AQI_O3'}, inplace=True)
NO2.rename(columns={'1st Max Value':'NO2','AQI':'AQI_NO2'}, inplace=True)
SO2.rename(columns={'1st Max Value':'SO2','AQI':'AQI_SO2'}, inplace=True)


In [ ]:
# Aggregate across POCs to get site-level data
def aggregate_site_level(df, mean_cols=[], max_cols=[]):
    agg_dict = {c:'mean' for c in mean_cols}
    agg_dict.update({c:'max' for c in max_cols})
    # Keep first values of metadata columns (latitude, longitude, names)
    meta_cols = ['Latitude','Longitude','State Name','City Name','County Name','Local Site Name']
    for col in meta_cols:
        if col in df.columns:
            agg_dict[col] = 'first'
    return df.groupby(['Date Local','Site Num','State Code','County Code'], as_index=False).agg(agg_dict)

PM_2_5_site = aggregate_site_level(PM_2_5, mean_cols=['PM25'], max_cols=['AQI_PM25'])
CO_site = aggregate_site_level(CO, max_cols=['CO','AQI_CO'])
O3_site = aggregate_site_level(O3, max_cols=['O3','AQI_O3'])
NO2_site = aggregate_site_level(NO2, max_cols=['NO2','AQI_NO2'])
SO2_site = aggregate_site_level(SO2, max_cols=['SO2','AQI_SO2'])


# Merge all pollutants on site-level keys
merge_cols = ['Date Local','Site Num','State Code','County Code']

merged = PM_2_5_site.merge(CO_site, on=merge_cols, how='inner')\
                     .merge(O3_site, on=merge_cols, how='inner')\
                     .merge(NO2_site, on=merge_cols, how='inner')\
                     .merge(SO2_site, on=merge_cols, how='inner')




In [ ]:
# Create site ID
merged['site_id'] = (merged['State Code'].astype(str).str.zfill(2) + '-' + merged['County Code'].astype(str).str.zfill(3) + '-' + merged['Site Num'].astype(str).str.zfill(4) )
merged['AQI'] = merged[['AQI_PM25', 'AQI_CO', 'AQI_O3', 'AQI_NO2', 'AQI_SO2']].max(axis=1)
# # Rename columns to match desired output
merged.rename(columns={
    'Date Local':'date',
    'Site Num':'site_num',
    'State Code':'state_code',
    'County Code':'county_code',
    'Latitude':'latitude',
    'Longitude':'longitude',
    'State Name':'state_name',
    'County Name':'county_name',
    'City Name':'city_name',
    'Local Site Name': 'site_name'

}, inplace=True)


# Columns to keep in order
cols = ['date','site_id','latitude', 'longitude','state_name','county_name','city_name','site_name','PM25','CO','O3','NO2','SO2', 'AQI_PM25', 'AQI_CO', 'AQI_O3', 'AQI_NO2', 'AQI_SO2','AQI' ]
merged = merged[cols]




In [170]:
merged = merged.dropna(subset=['AQI_O3'])


In [171]:
merged.isnull().sum()

date           0
site_id        0
latitude       0
longitude      0
state_name     0
county_name    0
city_name      0
site_name      0
PM25           0
CO             0
O3             0
NO2            0
SO2            0
AQI_PM25       0
AQI_CO         0
AQI_O3         0
AQI_NO2        0
AQI_SO2        0
AQI            0
dtype: int64

In [172]:
merged.groupby('state_name')['state_name'].count().sort_values(ascending=False)

state_name
California              2242
Pennsylvania            1514
Utah                    1446
Texas                    896
Virginia                 814
Ohio                     711
Arizona                  709
Maryland                 680
North Carolina           665
New Mexico               656
Nevada                   648
Oklahoma                 503
Florida                  459
New York                 418
Kentucky                 361
Georgia                  361
Connecticut              358
District Of Columbia     353
Missouri                 352
Wyoming                  351
New Hampshire            350
Iowa                     349
North Dakota             345
Colorado                 332
South Dakota             330
Vermont                  326
Alabama                  320
Oregon                   319
Washington               309
New Jersey               301
Hawaii                   289
Louisiana                288
Massachusetts            283
Minnesota                236
Ind

In [173]:
merged.shape

(19802, 19)

In [ ]:

# Save final site-level dataset
merged.to_csv('air_quality_sitelevel.csv', index=False)